In [1]:
from eval import evaluate_single_input
from datasets import load_dataset
import faiss
import numpy as np
import pickle
import openai
from openai import OpenAI

c:\Users\Hp\anaconda3\envs\cuda_deneme\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Initialize DeepInfra OpenAI Client
openai = OpenAI(
    api_key="2Q2AU9IG4jKLdqmRrHc2UxaLP8hHB0ii",  # Replace with your key
    base_url="https://api.deepinfra.com/v1/openai",
)

In [3]:
def load_faiss_index(index_path, metadata_path):
    """
    Loads the FAISS index and metadata from local storage.
    """
    index = faiss.read_index(index_path)
    with open(metadata_path, 'rb') as f:
        metadata = pickle.load(f)
    return index, metadata

In [4]:
def search_faiss(query, top_k, index_path, metadata_path):
    """
    Searches FAISS for the most relevant chunks to the query.
    """
    # Load index and metadata
    index, metadata = load_faiss_index(index_path, metadata_path)

    # Generate embedding for the query
    response = openai.embeddings.create(
        model="BAAI/bge-m3",
        input=query,
        encoding_format="float"
    )
    query_embedding = np.array([response.data[0].embedding])

    # Search FAISS index for the top_k results
    distances, indices = index.search(query_embedding, top_k)

    # Retrieve matching chunks
    results = []
    for i, idx in enumerate(indices[0]):
        results.append((metadata[idx], distances[0][i]))
    return results

In [5]:
# Load the dataset
dataset = load_dataset(
path='BlackFear/istanbul-qa-dataset',
trust_remote_code=True,
)

data = dataset['test']

# Extract data
queries = data['question']
references = data['reference']


c:\Users\Hp\anaconda3\envs\cuda_deneme\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Hp\.cache\huggingface\hub\datasets--BlackFear--istanbul-qa-dataset. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Generating test split: 100%|██████████| 300/300 [00:00<00:00, 74956.29 examples/s]


In [6]:
def generate_answer(query, context):
    """
    Generates an answer based on the query and retrieved context using GPT.
    """
    # Prepare the input prompt
    prompt = f"""
    Use the following contexts to answer the question:

    Context:
    {context}

    Question:
    {query}

    Answer:
    """

    # Query the Llama model
    response = openai.chat.completions.create(
        model="meta-llama/Llama-3.3-70B-Instruct-Turbo",
        messages=[
            {"role": "system", "content": "You will be answering questions about Istanbul. Please provide the answer to the following question."},
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content.strip()

In [7]:
def rag_pipeline(query, top_k, index_path, metadata_path):
    """
    Executes the RAG pipeline to retrieve relevant chunks and generate an answer.
    """
    # Step 1: Retrieve relevant chunks
    results = search_faiss(query, top_k, index_path, metadata_path)

    # Step 2: Combine the retrieved chunks as context
    context = "\n\n".join([f"Chunk {i+1}: {result[0]}" for i, result in enumerate(results)])

    # Step 3: Generate an answer using GPT
    answer = generate_answer(query, context)

    return answer, results

In [8]:
from tqdm import tqdm
scores = []
for i in tqdm(range(len(queries)), total=len(queries)):
    query = queries[i]
    reference = references[i]
    answer,results = rag_pipeline(query, top_k=3, index_path='faiss_index_semantic.index', metadata_path='metadata_semantic.pkl')
    results = [result[0] for result in results]
    score = evaluate_single_input(query, reference, answer, results)
    print(score)
    scores.append(score)

# 21m

  0%|          | 1/300 [00:06<33:55,  6.81s/it]

{'Faithfulness': 0.8, 'Relevancy': 0.8, 'Factual Correctness': 0.875, 'BLEU Score': 0.17464747586953588, 'Overall Score': 0.8300000000000001}


  1%|          | 2/300 [00:09<21:05,  4.25s/it]

{'Faithfulness': 0.0, 'Relevancy': 1.0, 'Factual Correctness': 1.0, 'BLEU Score': 0, 'Overall Score': 0.7}


  1%|          | 3/300 [00:14<22:24,  4.53s/it]

...

{'Faithfulness': 1.0, 'Relevancy': 0.9876, 'Factual Correctness': 0.875, 'BLEU Score': 0.2069016238132598, 'Overall Score': 0.94628}


100%|█████████▉| 299/300 [20:52<00:03,  3.80s/it]

{'Faithfulness': 0.9876, 'Relevancy': 0.92, 'Factual Correctness': 0.5, 'BLEU Score': 0.45454679014090354, 'Overall Score': 0.7722800000000001}


100%|██████████| 300/300 [20:59<00:00,  4.20s/it]

{'Faithfulness': 1.0, 'Relevancy': 0.9876, 'Factual Correctness': 0.8, 'BLEU Score': 0.08103230024642705, 'Overall Score': 0.91628}


In [9]:
faithfullness, relevance, factual, bleu, overall = 0, 0, 0, 0, 0
for i, score in enumerate(scores):
    faithfullness += scores[i]['Faithfulness']
    relevance += scores[i]['Relevancy']
    factual += scores[i]['Factual Correctness']
    bleu += scores[i]['BLEU Score']
    overall += scores[i]['Overall Score']

faithfullness /= len(scores)
relevance /= len(scores)
factual /= len(scores)
bleu /= len(scores)
overall /= len(scores)

In [10]:
print(f"Faithfullness: {faithfullness}")
print(f"Relevance: {relevance}")
print(f"Factual Correctness: {factual}")
print(f"BLEU Score: {bleu}")
print(f"Overall Score: {overall}")

"""Faithfullness: 0.7438866666666664
Relevance: 0.7978940000000004
Factual Correctness: 0.6772726666666667
BLEU Score: 0.21887150058996804
Overall Score: 0.733443266666666"""

Faithfullness: 0.7438866666666664
Relevance: 0.7978940000000004
Factual Correctness: 0.6772726666666667
BLEU Score: 0.21887150058996804
Overall Score: 0.733443266666666
